# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import time
import pandas as pd

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [2]:
driver = webdriver.Chrome()


In [3]:
time.sleep(1)
driver.get('https://jportal.mdcourts.gov/license/pbSearchResult.jsp?slcJurisdiction=50&txtTradeName=Vap%25&txtOwnerName=&txtLocationStreetName=&slcYear=2018&slcSortBy=ownername')

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [4]:
driver.get('https://jportal.mdcourts.gov/license/index_disclaimer.jsp')

Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [5]:
driver.find_element_by_id('checkbox').click()

In [6]:
driver.find_element_by_tag_name('form').submit()

In [7]:
driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [8]:
select = Select(driver.find_element_by_id('slcJurisdiction'))
select.select_by_visible_text('Statewide')

In [9]:
search_name = driver.find_element_by_id('txtTradeName')
search_name.send_keys('VAP%')

And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

In [10]:
driver.find_element_by_tag_name('form').submit()

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [11]:
for store in driver.find_elements_by_class_name('searchlistitem'):
    print(store.text)

VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [12]:
driver.find_elements_by_tag_name('a')[2].text

''

In [13]:
driver.find_elements_by_class_name('searchfieldtitle')[1].text

'2. VAPE IT STORE I'

In [14]:
#The above didn't seem particularly hard... but it's only iterating through that title field, preventing
#any further scraping...
'''driver.get('https://jportal.mdcourts.gov/license/index_disclaimer.jsp')
driver.find_element_by_id('checkbox').click()
driver.find_element_by_tag_name('form').submit()
driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')
    
select = Select(driver.find_element_by_id('slcJurisdiction'))
select.select_by_visible_text('Statewide')
'''
for store in driver.find_elements_by_class_name('searchfieldtitle'):
    print('------')
    if store.text[-9:] == 'Pending *':
        print(store.text.replace(' Pending *','')[3:])
        print('Pending')
    else:
        print(store.find_element_by_tag_name('img').get_attribute('alt').replace('Click for Detail of ',''))
        store.click()
        time.sleep(1)
        print(driver.find_element_by_class_name('tablecelltext').text)
        driver.back()
        time.sleep(2)
    print(store.text)
    


------
VAPE IT STORE II
  AMIN NARGIS Lic. Status: Issued


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.12.6 x86_64)


Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

In [17]:
'''driver.get('https://jportal.mdcourts.gov/license/index_disclaimer.jsp')
driver.find_element_by_id('checkbox').click()
driver.find_element_by_tag_name('form').submit()
driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')
    
select = Select(driver.find_element_by_id('slcJurisdiction'))
select.select_by_visible_text('Statewide')
'''
for store in driver.find_elements_by_class_name('searchfieldtitle'):
    print('------')
    if store.text[-9:] == 'Pending *':
        print(store.text.replace(' Pending *','')[3:])
        print('Pending')
    else:
        print(store.find_element_by_tag_name('img').get_attribute('alt').replace('Click for Detail of ',''))
        store.click()
        time.sleep(1)
        print(driver.find_element_by_class_name('tablecelltext').text)
        driver.back()
        time.sleep(2)
    print(store.text)

------
VAPE IT STORE II
AMIN NARGIS
1104 PLANTERS PLACE
SALISBURY, MD 21804 License Status: Issued
License No.: 22375606
Control No.: 22884439
Date of Issue: 4/27/2018
Months Paid: 12
Exp. Date: 4/30/2019
Subdivision: 22 Salisbury


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.12.6 x86_64)


### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [18]:
vape_list = []
for store in driver.find_elements_by_class_name('searchfieldtitle'):
    vape_dict = {}
    print('------')
    if store.text[-9:] == 'Pending *':
        print(store.text.replace(' Pending *','')[3:])
        print('Pending')
        vape_dict['name'] = store.text.replace(' Pending *','')[3:]
        vape_dict['url'] = ''
    else:
        print(store.find_element_by_tag_name('img').get_attribute('alt').replace('Click for Detail of ',''))
        vape_dict['name'] = store.find_element_by_tag_name('img').get_attribute('alt').replace('Click for Detail of ','')
        vape_dict['url'] = store.find_element_by_tag_name('a').get_attribute('href')
    vape_list.append(vape_dict)

df = pd.DataFrame(vape_list)
df
        

------
VAPE IT STORE II
------
VAPE IT STORE I
------
VAPEPAD THE
------
VAPE FROG
Pending
------
VAPE FROG
Pending


,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE FROG,
4,VAPE FROG,


### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [19]:
driver.get('https://jportal.mdcourts.gov/license/index_disclaimer.jsp')
driver.find_element_by_id('checkbox').click()
driver.find_element_by_tag_name('form').submit()
driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')
search_name = driver.find_element_by_id('txtTradeName')
search_name.send_keys('VAP%')
select = Select(driver.find_element_by_id('slcJurisdiction'))
select.select_by_visible_text('Statewide')
driver.find_element_by_tag_name('form').submit()


next_button = driver.find_element_by_partial_link_text('Next ')
driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
next_button.click()

### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [20]:
driver.get('https://jportal.mdcourts.gov/license/index_disclaimer.jsp')
driver.find_element_by_id('checkbox').click()
driver.find_element_by_tag_name('form').submit()
driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')
search_name = driver.find_element_by_id('txtTradeName')
search_name.send_keys('VAP%')
select = Select(driver.find_element_by_id('slcJurisdiction'))
select.select_by_visible_text('Statewide')
driver.find_element_by_tag_name('form').submit()

while True:
    next_button = driver.find_element_by_partial_link_text('Next ')
    driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
    next_button.click()
    print('Scraping a new page')
    

Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next "}
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.12.6 x86_64)


### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [21]:
driver.get('https://jportal.mdcourts.gov/license/index_disclaimer.jsp')
driver.find_element_by_id('checkbox').click()
driver.find_element_by_tag_name('form').submit()
driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')
search_name = driver.find_element_by_id('txtTradeName')
search_name.send_keys('VAP%')
select = Select(driver.find_element_by_id('slcJurisdiction'))
select.select_by_visible_text('Statewide')
driver.find_element_by_tag_name('form').submit()

vape_list = []

try:
    while True:
        for store in driver.find_elements_by_class_name('searchlistitem'):
            print(store.text)
            vape_list.append(store.text)
        next_button = driver.find_element_by_partial_link_text('Next ')
        driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
        next_button.click()
        print('Scraping a new page')
except:
    print('Last page folks')



VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG
Scraping a new page
VAPE LOFT THE
VAPE N CIGAR
VAPE DOJO
VAPE HAVEN
VAPORS LOUNGE
Scraping a new page
VAPORFI SOUTHSIDE
VAPE DO JO
VAPERTETMD
VAPERTEKMD
VAPORIOT
Scraping a new page
VAPE PLUS
VAPE SOCIAL
VAPE JUNGLE
VAPE TIME
VAPEBAR EXPESS
Scraping a new page
VAPEWERKS
VAPIST MD
VAPOR 99
VAPOR ALLEY
VAPOR EMPORIUM
Scraping a new page
VAPOR VILLA
VAPOR VILLA
VAPORS LOUNGE
VAPE DOJO
VAPE DOJO
Scraping a new page
VAPE DOJO
VAPE DOJO
VAPEZ YARDHOUSE
VAPERS RING
Last page folks


In [22]:
vape_list = []

try:
    while True:
        for store in driver.find_elements_by_class_name('searchfieldtitle'):
            vape_dict = {}
            print('-------------')
            if store.text[-9:] == 'Pending *':
                print(store.text.replace(' Pending *','')[3:])
                print('Pending')
                vape_dict['name'] = store.text.replace(' Pending *','')[3:]
                vape_dict['url'] = ''
                vape_list.append(vape_dict)
            else:
                print(store.find_element_by_tag_name('img').get_attribute('alt').replace('Click for Detail of ',''))
                vape_dict['name'] = store.find_element_by_tag_name('img').get_attribute('alt').replace('Click for Detail of ','')
                vape_dict['url'] = store.find_element_by_tag_name('a').get_attribute('href') 
                vape_list.append(vape_dict)
        next_button = driver.find_element_by_partial_link_text('Next ')
        driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
        next_button.click()
        print('Scraping a new page')
except:
    print('Last page folks')

-------------
VAPE IT STORE II
-------------
VAPE IT STORE I
-------------
VAPEPAD THE
-------------
VAPE FROG
Pending
-------------
VAPE FROG
Pending
Scraping a new page
-------------
VAPE LOFT THE
-------------
VAPE N CIGAR
-------------
VAPE DOJO
-------------
VAPE HAVEN
Pending
-------------
 VAPORS LOUNGE
Pending
Scraping a new page
-------------
VAPORFI SOUTHSIDE
-------------
VAPE DO JO
-------------
 VAPERTETMD
Pending
-------------
VAPERTEKMD
-------------
 VAPORIOT
Pending
Scraping a new page
-------------
VAPE PLUS
-------------
VAPE  SOCIAL
-------------
VAPE JUNGLE
-------------
 VAPE TIME
Pending
-------------
 VAPEBAR EXPESS
Pending
Scraping a new page
-------------
VAPEWERKS
-------------
VAPIST MD
-------------
 VAPOR 99
Pending
-------------
VAPOR ALLEY
-------------
VAPOR EMPORIUM
Scraping a new page
-------------
VAPOR VILLA
-------------
VAPOR VILLA
-------------
VAPORS LOUNGE
-------------
VAPE DOJO
-------------
VAPE DOJO
Scraping a new page
-------------
VAPE DO

In [23]:
vape_list

[{'name': 'VAPE IT STORE II',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pUdvbOe9%2BNY%3D'},
 {'name': 'VAPE IT STORE I',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=j5g0Oi3UWXk%3D'},
 {'name': 'VAPEPAD THE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=K81VUcRi3L0%3D'},
 {'name': 'VAPE FROG', 'url': ''},
 {'name': 'VAPE FROG', 'url': ''},
 {'name': 'VAPE LOFT THE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=nCgDu4%2BQX7o%3D'},
 {'name': 'VAPE N CIGAR',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ZOLktHBqjFI%3D'},
 {'name': 'VAPE DOJO',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=%2FHTYPdv0S9M%3D'},
 {'name': 'VAPE HAVEN', 'url': ''},
 {'name': ' VAPORS LOUNGE', 'url': ''},
 {'name': 'VAPORFI SOUTHSIDE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=u25juCqYM94%3D'},
 {'name': 'VAPE DO JO',
  'url': '

### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [24]:
df = pd.DataFrame(vape_list)

In [25]:
df.to_csv('vape_shops_basic.csv', index=False)

# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [26]:
pd.set_option('display.max_colwidth', -1)

In [27]:
df.url[0]

'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pUdvbOe9%2BNY%3D'

### Use Selenium to visit that page

In [28]:
driver.get(df.url[0])

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [29]:
vape_dict = {}
vape_dict['mailing_address'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text
vape_dict['location_address'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
vape_dict['license_info'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text.split('\n')
#vape_dict['total_amount_paid'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr[8]/td[2]').text
vape_dict['issued_by'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text
vape_dict['licenses'] = [link.text for link in driver.find_elements_by_tag_name('a')[10:-16]]
vape_dict['total_amount_paid'] = driver.find_elements_by_class_name('rt')[-1].text
vape_dict

{'mailing_address': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804',
 'location_address': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801',
 'license_info': ['License Status: Issued',
  'License No.: 22375606',
  'Control No.: 22884439',
  'Date of Issue: 4/27/2018',
  'Months Paid: 12',
  'Exp. Date: 4/30/2019',
  'Subdivision: 22 Salisbury'],
 'issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198',
 'licenses': ['OTHER TOBACCO PROD RETLR', 'CHAIN STORE', "TRADER'S LICENSE"],
 'total_amount_paid': '$ 41.00'}

### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

In [48]:


driver.get(df.url[2]) #Obvi we'll change the input here shortly.

vape_dict = {}
vape_dict['mailing_address'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text
vape_dict['location_address'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
vape_dict['license_info'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text.split('\n')
vape_dict['issued_by'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text
vape_dict['licenses'] = [link.text for link in driver.find_elements_by_tag_name('a')[10:-16]]
vape_dict['total_amount_paid'] = driver.find_elements_by_class_name('rt')[-1].text

vape_dict

{'mailing_address': 'ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\nODENTON, MD 21113',
 'location_address': 'VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILLS, MD 21054',
 'license_info': ['License Status: Issued',
  'License No.: 02304705',
  'Control No.: 02685930',
  'Date of Issue: 4/05/2018',
  'Months Paid: 12',
  'Exp. Date: 4/30/2019',
  'Subdivision: 02 Anne Arundel County'],
 'issued_by': 'ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401',
 'licenses': ["TRADER'S LICENSE"],
 'total_amount_paid': '$ 94.00'}

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [40]:
df.url[1]

'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=j5g0Oi3UWXk%3D'

In [74]:
def vape_scrape(row):
    try:
        driver.get(row.url) 
        time.sleep(1)
        vape_dict = {}
        vape_dict['mailing_address'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text
        vape_dict['location_address'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
        vape_dict['license_info'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text.split('\n')
        vape_dict['issued_by'] = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[3]/tbody/tr[2]/td').text
        vape_dict['licenses'] = [link.text for link in driver.find_elements_by_tag_name('a')[10:-16]]
        vape_dict['total_amount_paid'] = driver.find_elements_by_class_name('rt')[-1].text
        return pd.Series(vape_dict)
    except:
        return pd.Series({})

### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [86]:
df_test = df.head(3).apply(vape_scrape, axis=1).join(df)

In [87]:
df_test

,mailing_address,location_address,license_info,issued_by,licenses,total_amount_paid,name,url
0,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804","VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801","[License Status: Issued, License No.: 22375606, Control No.: 22884439, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]","WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198","[OTHER TOBACCO PROD RETLR, CHAIN STORE, TRADER'S LICENSE]",$ 41.00,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pUdvbOe9%2BNY%3D
1,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804","VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801","[License Status: Issued, License No.: 22375605, Control No.: 22591855, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]","WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198","[TRADER'S LICENSE, CHAIN STORE]",$ 24.00,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=j5g0Oi3UWXk%3D
2,"ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\nODENTON, MD 21113","VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILLS, MD 21054","[License Status: Issued, License No.: 02304705, Control No.: 02685930, Date of Issue: 4/05/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 02 Anne Arundel County]","ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401",[TRADER'S LICENSE],$ 94.00,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=K81VUcRi3L0%3D


In [89]:
#So while it seems to work, it won't capture any data on the pending applications. I'll leave it out for now.
#If we did want to grab their addresses we'd have to loop through the search result pages as above.

df_vape = df.apply(vape_scrape, axis=1).join(df)

## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.

In [92]:
df_vape.to_csv('vape-total.csv', index=False)